In [1]:
from langchain_groq import ChatGroq
llm=ChatGroq(
    temperature=0,
    groq_api_key='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',
    model_name="llama3-70b-8192"
    )

response=llm.invoke("The first human on moon")
print(response.content)



You're referring to Neil Armstrong!

Neil Armstrong was an American astronaut, engineer, and naval aviator who became the first person to walk on the Moon on July 20, 1969, during the Apollo 11 mission.

Armstrong was born on August 5, 1930, in Wapakoneta, Ohio. He developed an interest in flight at a young age and earned his pilot's license at 15. He studied aeronautical engineering at Purdue University and later served in the United States Navy as a naval aviator during the Korean War.

In 1962, Armstrong was selected as an astronaut by NASA, and he flew on the Gemini 8 mission in 1966. Three years later, he was chosen to command the Apollo 11 mission, which aimed to land humans on the Moon.

On July 16, 1969, Armstrong, along with Edwin "Buzz" Aldrin and Michael Collins, launched into space aboard the Saturn V rocket. After traveling through space for four days, the Apollo 11 spacecraft entered into lunar orbit. On July 20, Armstrong and Aldrin suited up in their spacesuits and clim

In [2]:
import os
os.environ["USER_AGENT"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 Chrome/122.0.0.0 Safari/537.36"

from langchain_community.document_loaders import WebBaseLoader

url = "https://careers.nike.com/lead-software-engineer/job/R-60256"
loader = WebBaseLoader(url)
docs = loader.load()

print(docs[0].page_content)























Lead Software Engineer










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu



Sel

In [3]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("""
    "The scraped job description is stored in the variable 'docs'. Extract the following information in JSON format:\n\n"
    "- role_type: The job title\n"
    "- description: A concise summary of responsibilities\n"
    "- skills_required: A list of required or preferred skills\n\n"
    "Text:\n{docs}"
    """)

chain_extract=prompt_template|llm
res=chain_extract.invoke(input={'docs':docs})
print (res.content)

Here is the extracted information in JSON format:

```
{
  "role_type": "Lead Software Engineer",
  "description": "Develop, code, configure, and test programs, systems, and solutions to meet designed digital product specifications and direction; manage the team in gathering detailed designs and solving complex problems;",
  "skills_required": [
    "Python",
    "Amazon Web Services",
    "AWS Sagemaker",
    "Terraform",
    "Snowflake",
    "Streamlit",
    "Postgres",
    "Kubernetes",
    "Docker",
    "Databricks",
    "Test-driven development",
    "Large language models",
    "Jenkins"
  ]
}
```

Note that the description is a concise summary of the responsibilities, and the skills required are extracted from the job description.


In [4]:
from langchain_core.output_parsers import JsonOutputParser
json_parser=JsonOutputParser()
json_res=json_parser.parse(res.content)
print(json_res)


{'role_type': 'Lead Software Engineer', 'description': 'Develop, code, configure, and test programs, systems, and solutions to meet designed digital product specifications and direction; manage the team in gathering detailed designs and solving complex problems;', 'skills_required': ['Python', 'Amazon Web Services', 'AWS Sagemaker', 'Terraform', 'Snowflake', 'Streamlit', 'Postgres', 'Kubernetes', 'Docker', 'Databricks', 'Test-driven development', 'Large language models', 'Jenkins']}


In [5]:
import pandas as pd
df=pd.read_csv("app/resource/sample_portfolio_links.csv")


In [6]:
import chromadb
import uuid
client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(
            documents=[row["Techstack"]],
            metadatas={"links": row["Links"]},
            ids=[str(uuid.uuid4())]
        )


In [7]:
job=json_res
job['skills_required']

['Python',
 'Amazon Web Services',
 'AWS Sagemaker',
 'Terraform',
 'Snowflake',
 'Streamlit',
 'Postgres',
 'Kubernetes',
 'Docker',
 'Databricks',
 'Test-driven development',
 'Large language models',
 'Jenkins']

In [8]:
# Join all links into a string formatted as bullet points
links = "\n".join(f"- {link}" for link in df["Links"])


prompt_email = PromptTemplate.from_template(

"""
### JOB DESCRIPTION:
{job_description}

### INSTRUCTION:
You are Priya, a **Business Executive** at InsightIQ, a consulting firm that connects companies with highly skilled technology professionals for short-term and long-term engagements.

You recently reviewed the job description above and identified several suitable candidates from your talent pool.

Each candidate’s portfolio is listed under a specific tech stack in the following links:
{link_list}

Your task is to write a cold email to the client regarding the job opportunity above. Highlight how InsightIQ can help them fulfill their hiring needs by showcasing candidate portfolios that match the job’s required skills.

Clearly state how InsightIQ can streamline their hiring process and why these candidates are a strong fit.

Remember you are Priya, Business Executive at InsightIQ.  
Do not provide a preamble.  
### EMAIL (NO PREAMBLE):
"""


)
chain_email=prompt_email|llm
res = chain_email.invoke({
    "job_description": str(job),
    "link_list": links
})
print(res.content)


Subject: Expert Lead Software Engineer Candidates for Your Digital Product Development

Dear [Client],

I came across the job description for a Lead Software Engineer role at your esteemed organization, and I'm excited to introduce you to a pool of highly skilled technology professionals from InsightIQ's talent network. Our candidates possess the exact skills you're looking for to develop, code, configure, and test programs, systems, and solutions to meet your designed digital product specifications.

After reviewing your job requirements, I've shortlisted candidates with impressive portfolios that align with your needs. Specifically, I'd like to highlight the following candidates:

* [Candidate 1] with expertise in Python, Amazon Web Services, and Terraform, showcasing their skills in building scalable and efficient systems (https://example.com/python-portfolio).
* [Candidate 2] with a strong background in AWS Sagemaker, Snowflake, and Streamlit, demonstrating their ability to design 